In [1]:
import pandas as pd
import string
import nltk
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, precision_recall_curve, accuracy_score
from sklearn.metrics import roc_auc_score
# !pip install imblearn --quiet
# !pip install llm2vec --quiet
# !pip install flash-attn --no-build-isolation
# import torch
# from llm2vec import LLM2Vec
import pickle


import warnings

warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data preprocessing

In [3]:
test_data = pd.read_csv('/content/drive/MyDrive/Agent_LLM_paper/drugsComTest_raw.tsv',sep = '\t')
train_data = pd.read_csv('/content/drive/MyDrive/Agent_LLM_paper/drugsComTrain_raw.tsv',sep = '\t')
test_data['label'] = test_data['rating'].apply(lambda x: 0 if x <= 5 else 1)
train_data['label'] = train_data['rating'].apply(lambda x: 0 if x <= 5 else 1)

In [4]:
y_train, y_test = train_data['label'], test_data['label']

In [5]:
with open('/content/drive/MyDrive/Agent_LLM_paper/x_train_infersent.pkl', 'rb') as fichier:
    x_train = pickle.load(fichier)

with open('/content/drive/MyDrive/Agent_LLM_paper/x_test_infersent.pkl', 'rb') as fichier:
    x_test = pickle.load(fichier)

# 1. RandomForestClassifier

In [6]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0)

In [7]:
model.fit(x_train, y_train)

RandomForestClassifier(random_state=0)

In [8]:
model.score(x_train, y_train)

0.9998884046200487

In [9]:
y_pred = model.predict(x_test)

In [10]:
target_names = ['negative', 'positive']
y_predictions = (y_pred > 0.5).astype(int)
predictions = pd.DataFrame(y_predictions, columns=['Prediction'])
predictions.reset_index(inplace=True)
predictions.columns = ['Index', 'Prediction']
auc = roc_auc_score(y_test, y_pred)
print(f"L'AUC est de : {auc}")
print(classification_report(y_test, predictions['Prediction'], target_names=target_names))

L'AUC est de : 0.8162826579638928
              precision    recall  f1-score   support

    negative       0.98      0.64      0.77     16207
    positive       0.86      0.99      0.92     37559

    accuracy                           0.89     53766
   macro avg       0.92      0.82      0.85     53766
weighted avg       0.90      0.89      0.88     53766



# 2. ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model4 = ExtraTreesClassifier(random_state=0)

In [ ]:
model4.fit(x_train, y_train)

ExtraTreesClassifier(random_state=0)

In [ ]:
model4.score(x_train, y_train)

0.9998884046200487

In [ ]:
y_pred = model4.predict(x_test)

In [ ]:
target_names = ['negative', 'positive']
y_predictions = (y_pred > 0.5).astype(int)
predictions = pd.DataFrame(y_predictions, columns=['Prediction'])
predictions.reset_index(inplace=True)
predictions.columns = ['Index', 'Prediction']
auc = roc_auc_score(y_test, y_pred)
print(f"L'AUC est de : {auc}")
print(classification_report(y_test, predictions['Prediction'], target_names=target_names))

L'AUC est de : 0.8125659774155819
              precision    recall  f1-score   support

    negative       0.99      0.63      0.77     16207
    positive       0.86      1.00      0.92     37559

    accuracy                           0.89     53766
   macro avg       0.93      0.81      0.85     53766
weighted avg       0.90      0.89      0.88     53766

L'AUC est de : 0.8125659774155819
              precision    recall  f1-score   support

    negative       0.99      0.63      0.77     16207
    positive       0.86      1.00      0.92     37559

    accuracy                           0.89     53766
   macro avg       0.93      0.81      0.85     53766
weighted avg       0.90      0.89      0.88     53766



# 3. Bi-LSTM

In [ ]:
x_train2 = np.expand_dims(x_train, axis=1)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout

embedding_dim = 4096
num_classes = 2

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, return_sequences=True),input_shape=(1, embedding_dim)))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(80, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x_train2, y_train, epochs=5)

Epoch 1/5
5041/5041 [==============================] - 92s 17ms/step - loss: 0.4908 - accuracy: 0.7676
Epoch 2/5
5041/5041 [==============================] - 87s 17ms/step - loss: 0.4509 - accuracy: 0.7909
Epoch 3/5
5041/5041 [==============================] - 86s 17ms/step - loss: 0.4353 - accuracy: 0.7996
Epoch 4/5
5041/5041 [==============================] - 87s 17ms/step - loss: 0.4235 - accuracy: 0.8047
Epoch 5/5
5041/5041 [==============================] - 86s 17ms/step - loss: 0.4129 - accuracy: 0.8115


In [ ]:
x_test2 = np.expand_dims(x_test, axis=1)

In [ ]:
y_pred = model.predict(x_test2)


1681/1681 [==============================] - 8s 4ms/step


In [ ]:
target_names = ['negative', 'positive']
y_predictions = (y_pred > 0.5).astype(int)
predictions = pd.DataFrame(y_predictions, columns=['Prediction'])
predictions.reset_index(inplace=True)
predictions.columns = ['Index', 'Prediction']
auc = roc_auc_score(y_test, y_pred)
print(f"L'AUC est de : {auc}")
print(classification_report(y_test, predictions['Prediction'], target_names=target_names))

L'AUC est de : 0.8558511088519798
              precision    recall  f1-score   support

    negative       0.69      0.63      0.66     16207
    positive       0.85      0.88      0.86     37559

    accuracy                           0.81     53766
   macro avg       0.77      0.76      0.76     53766
weighted avg       0.80      0.81      0.80     53766



# 4. DNN

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


input_dim = 4096
output_dim = 2

model = Sequential()


model.add(Dense(256, activation='relu', input_shape=(input_dim,)))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
5041/5041 [==============================] - 32s 6ms/step - loss: 0.5360 - accuracy: 0.7276
Epoch 2/5
5041/5041 [==============================] - 31s 6ms/step - loss: 0.4934 - accuracy: 0.7549
Epoch 3/5
5041/5041 [==============================] - 31s 6ms/step - loss: 0.4796 - accuracy: 0.7633
Epoch 4/5
5041/5041 [==============================] - 31s 6ms/step - loss: 0.4680 - accuracy: 0.7716
Epoch 5/5
5041/5041 [==============================] - 31s 6ms/step - loss: 0.4627 - accuracy: 0.7725


In [13]:
y_pred = model.predict(x_test)

1681/1681 [==============================] - 3s 2ms/step


In [14]:
target_names = ['negative', 'positive']
y_predictions = (y_pred > 0.5).astype(int)
predictions = pd.DataFrame(y_predictions, columns=['Prediction'])
predictions.reset_index(inplace=True)
predictions.columns = ['Index', 'Prediction']
auc = roc_auc_score(y_test, y_pred)
print(f"L'AUC est de : {auc}")
print(classification_report(y_test, predictions['Prediction'], target_names=target_names))

L'AUC est de : 0.8249167879614702
              precision    recall  f1-score   support

    negative       0.75      0.46      0.57     16207
    positive       0.80      0.93      0.86     37559

    accuracy                           0.79     53766
   macro avg       0.78      0.70      0.72     53766
weighted avg       0.79      0.79      0.78     53766



# 5. CNN

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(embedding_dim,1)))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(80, activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
x_train_reshaped = x_train.reshape(-1, embedding_dim, 1)

In [ ]:
model.fit(x_train_reshaped, y_train, epochs=5)

Epoch 1/5
5041/5041 ━━━━━━━━━━━━━━━━━━━━ 120s 22ms/step - accuracy: 0.7221 - loss: 0.5639
Epoch 2/5
5041/5041 ━━━━━━━━━━━━━━━━━━━━ 110s 22ms/step - accuracy: 0.7706 - loss: 0.4800
Epoch 3/5
5041/5041 ━━━━━━━━━━━━━━━━━━━━ 110s 22ms/step - accuracy: 0.7842 - loss: 0.4597
Epoch 4/5
5041/5041 ━━━━━━━━━━━━━━━━━━━━ 110s 22ms/step - accuracy: 0.7903 - loss: 0.4476
Epoch 5/5
5041/5041 ━━━━━━━━━━━━━━━━━━━━ 110s 22ms/step - accuracy: 0.7945 - loss: 0.4381


In [ ]:
x_test2 = x_test.reshape(-1, embedding_dim, 1)

In [ ]:
y_pred = model.predict(x_test2)

1681/1681 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step


In [ ]:
target_names = ['negative', 'positive']
y_predictions = (y_pred > 0.5).astype(int)
predictions = pd.DataFrame(y_predictions, columns=['Prediction'])
predictions.reset_index(inplace=True)
predictions.columns = ['Index', 'Prediction']
auc = roc_auc_score(y_test, y_pred)
print(f"L'AUC est de : {auc}")
print(classification_report(y_test, predictions['Prediction'], target_names=target_names))

L'AUC est de : 0.8481100399487801
              precision    recall  f1-score   support

    negative       0.73      0.53      0.61     16207
    positive       0.82      0.92      0.86     37559

    accuracy                           0.80     53766
   macro avg       0.78      0.72      0.74     53766
weighted avg       0.79      0.80      0.79     53766

